In [1]:
import json
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from soynlp.hangle import jamo_levenshtein
from tqdm import tqdm

from icecream import ic

with open("./jasa_dict.json", "r") as f:
    base_dict = json.load(f)
    
def convert_keys_to_int(d):
    if isinstance(d, dict):
        new_dict = {}
        for key, value in d.items():
            # 문자열 키를 int로 변환 (가능할 경우에만)
            new_key = int(key) if key.isdigit() else key
            new_dict[new_key] = convert_keys_to_int(value)
        return new_dict
    elif isinstance(d, list):
        return [convert_keys_to_int(item) for item in d]
    else:
        return d

In [109]:
base_dict = convert_keys_to_int(base_dict)

In [110]:
def overlap_checking(word, comname):
    # word = word1.replace(' ', '')
    word = word.replace(comname, '#%#')
    rep_count = word.count('#%#')
    word = " " * len(comname) + word
    comname_leng = len(comname)
    word_leng = len(word)
    count_lst = []
    for i in range(word_leng - comname_leng + 1):
        cnt = 0
        for let_a, let_b in zip(word[i:i + comname_leng], comname):
            if let_a == let_b:
                cnt += 1
        count_lst.append(cnt)
    # print(count_lst)
    max_args = np.argmax(count_lst)
    result = word[max_args:max_args + comname_leng]
    if len(result) > len(word):
        return word, max(count_lst)
    return result, max(count_lst)

# def overlap_checking(word, comname):
#     # word = word1.replace(' ', '')
#     word_origin = word[:]
#     comname_origin = comname[:]
#     word = word.upper()
#     comname = comname.upper()
    
#     word = word.replace(comname, '#%#')
#     rep_count = word.count('#%#')
#     word = " " * len(comname) + word
#     comname_leng = len(comname)
#     word_leng = len(word)
#     count_lst = []
#     for i in range(word_leng - comname_leng + 1):
#         cnt = 0
#         for let_a, let_b in zip(word[i:i + comname_leng], comname):
#             if let_a == let_b:
#                 cnt += 1
#         count_lst.append(cnt)
#     # print(count_lst)
#     max_args = np.argmax(count_lst)
#     result = word_origin[max_args:max_args + comname_leng]
    
#     if len(result) > len(word):
#         return word_origin, max(count_lst)
#     return result, max(count_lst)


def find_word_indices(sent1, word1):
    indices = []
    index = sent1.find(word1)
    while index != -1:
        indices.append(index + 1)
        index = sent1.find(word1, index + 1)
    return indices

def main(comname, base_dict):
    ##### 과업 내용 출력
    print("#"*50)
    print("자사명 오기재 검출 중 : jasamyeong.py")
    print("#"*50)
    #####
    
    tokenizer = Okt()
    output_dict = {}
    # comname = args.company_name
    com_leng = len(comname)
    max_leven = jamo_levenshtein(' ' * com_leng, comname)
    for user, rows in tqdm(base_dict.items()):
        if user not in output_dict:
            output_dict[user] = {}
        for col, docs in rows.items():
            if col not in output_dict[user]:
                output_dict[user][col] = {}
            final_lst = []
            for sent_ind, sent in docs.items():
                # sent_origin = sent[:]
                # comname_origin = comname[:]
                # sent = sent.upper()
                # comname = comname.upper()
                
                mini_dict = {'sent_ind': sent_ind + 1, 'sent': sent}
                ov_ck = overlap_checking(sent, comname)
                leven_ov = jamo_levenshtein(ov_ck[0], comname)
                print(sent, comname)
                print(ov_ck)
                print(leven_ov)

                if leven_ov < 0.7:
                    test_word = ''.join([item[0] for item in tokenizer.pos(ov_ck[0]) if item[1] != 'Josa'])
                    # print(user, col, sent_ind, '|', f'[{test_word}] | {sent}')
                    mini_dict['word_ind'] = find_word_indices(sent, test_word)
                    mini_dict['key_word'] = test_word
                    final_lst.append(mini_dict)
                elif com_leng >= 4 and (ov_ck[1] == com_leng - 2 and ' ' not in ov_ck[0] and leven_ov < 1):
                    test_word = ''.join([item[0] for item in tokenizer.pos(ov_ck[0]) if item[1] != 'Josa'])
                    # print(user, col, sent_ind, '|', f'[{test_word}] | {sent}')
                    mini_dict['word_ind'] = find_word_indices(sent, test_word)
                    mini_dict['key_word'] = test_word
                    final_lst.append(mini_dict)
                elif com_leng >= 6 and ov_ck[1] == com_leng - 3 and ' ' in ov_ck[0]:
                    new_keys = max(ov_ck[0].split(' '), key=len)
                    jamo_ov_ck = overlap_checking(new_keys, comname)
                    leven_jamo = jamo_levenshtein(new_keys, comname)
                    if jamo_ov_ck[1] >= com_leng - 3 and leven_jamo < max_leven // 2:
                        # print(user, col, sent_ind, '|', f'[{new_keys}] | {sent}')
                        mini_dict['word_ind'] = find_word_indices(sent, new_keys)
                        mini_dict['key_word'] = new_keys
                        final_lst.append(mini_dict)
                if final_lst != []:
                    output_dict[user][col] = final_lst
                else:
                    output_dict[user][col] = 0
    return output_dict

def dict_transform(output_dict):
    print('Dict Transforming...')
    transformed_data = {}
    for key, inner_dict in tqdm(output_dict.items()):
        inner_data = {}
        for inner_key, values_dict in inner_dict.items():
            inner_data[f'자사명 오기재 검출{inner_key + 1}'] = json.dumps(values_dict, ensure_ascii=False)
        transformed_data[key] = inner_data
    countdf = pd.DataFrame.from_dict(transformed_data, orient='index')
    
    ##### "자사명 오기재 검출n" -> "검출n"
    
    countdf.columns = ["검출" + i[-1] for i in ['자기소개서1', '자기소개서2', '자기소개서3', '자기소개서4', '자기소개서5', '자기소개서6']]
    #####
    
    return countdf

In [111]:
output_dict = main('한전KPS', base_dict)
# finaldf = dict_transform(output_dict)

##################################################
자사명 오기재 검출 중 : jasamyeong.py
##################################################


100%|████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 234.85it/s]

한전LPS에 들어가고 싶습니다. 한전KPS
('한전LPS', 4)
1
뉴스를 통해 ‘2030, 2050 탄소 중립’을 접하게 되었고 단순하게 생각했던 자연재해나 이상기온의 원인이 일회용품과 화석연료에서 배출되는 탄소 때문에 일어나고 있다는 것을 알게 되었습니다. 한전KPS
('     ', 0)
5
배출 온실가스의 대부분이 에너지 소비과정에서 발생하고 있으므로 에너지 분야는 탄소 중립 실현의 핵심 분야라고 생각합니다. 한전KPS
('     ', 0)
5
그 중 한전kps는 안전한 전력공급과 함께 전기차충전사업, 태양광발전사업 등의 에너지신사업을 주도하며 친환경적인 움직임을 보이고 있습니다. 한전KPS
('한전kps', 2)
3
0000과학기술원에서 연구본부지원실에 소속해 업무추진비 지급신청과 지출 관리의 회계 업무를 담당하였습니다. 한전KPS
('     ', 0)
5
이때 문제해결역량을 기를 수 있었습니다. 한전KPS
('     ', 0)
5
증빙서류 누락이나 잘못 기재되는 등 재요청을 하는 건수를 개선안을 마련하여 한 달에 8회에서 0건으로 만들었습니다. 한전KPS
('한 달에 ', 1)
4
이때 통신비 납부 지연으로 인해 발생한 수수료 300원을 납부 방법변경으로 미발생하게 하였습니다. 한전KPS
('한 수수료', 1)
4
우리나라는 해외에 비해 친환경 에너지를 사용하기에 기술적, 지형적문제가 있습니다. 한전KPS
('     ', 0)
5
이를 극복하기위한 방면으로는 그린 뉴딜 정책이 가장 적합하다고 생각하며 한국에 맞는 운영을 할 때 더욱 효율적인 운영이 필요하고 이를 위해서 디지털화가 중심이 되어야 한다고 생각합니다. 한전KPS
('한 방면으', 1)
4
디지털 뉴딜을 통해 공공일자리를 넘어 숙련된 질높은 일자리를 구축하는 등 구조적 변화를 해야합니다. 한전KPS
('     ', 0)
5
이를 시행할 때 있어서 무엇보다 안전한 전력수급이 중요합니다. 한전KPS
('안전한 전', 1)
3.333333333333333
이를 위해서는 전력과 디지털 

In [112]:
output_dict

{0: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0},
 1: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0},
 2: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0},
 3: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0},
 4: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0},
 5: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0},
 6: {0: 0, 1: 0, 2: 0, 3: 0, 4: 0, 5: 0}}

In [32]:
a = pd.read_excel("./data/jasamyeong_data/자사명 등록 양식0905.xlsx")
a

,한전KPS,한전kps,한전
0,NaN,대한전화,NaN


In [139]:
from rapidfuzz import fuzz
from fuzzywuzzy import process

# 기본 비교 대상인 텍스트
target_word = "한전KPS"

# 검출하고자 하는 문장
sentence = "한전KSP는 이번 프로젝트에 참여하지 않았지만, 한전KPS는 주요 참여자입니다."

# 문장 내에서 단어 리스트로 분리
words = sentence.split()

# 각 단어와 target_word의 유사도 측정
similar_words = [(word, fuzz.ratio(word, target_word)) for word in words]
print(similar_words)
# 유사도가 80 이상인 단어들만 필터링 (유사도 임계치는 상황에 따라 조정 가능)
similar_words = [word for word, score in similar_words if score >= 80]

print(similar_words)


[('한전KSP는', 72.72727272727273), ('이번', 0.0), ('프로젝트에', 0.0), ('참여하지', 0.0), ('않았지만,', 0.0), ('한전KPS는', 90.9090909090909), ('주요', 0.0), ('참여자입니다.', 0.0)]
['한전KPS는']


In [288]:
query_jamo

'ㅎㅏㄴㅈㅓㄴKPS'

In [289]:
choices_jamo

['ㅎㅏㄴㅈㅓㄴLPS',
 'ㅎㅏㄴㅈㅓㄴㄷㅗㄹㅗㄱㅗㅇㅅㅏ',
 'ㅎㅏㄴㅈㅓㄴICPS',
 'ㅎㅏㄴㄱㅜㄱㅈㅓㄴㄹㅕㄱKPS',
 'KPSㅎㅏㄴㅈㅓㄴ',
 'ㅎㅏㄴㄱㅜㄱKPS',
 'ㅈㅓㄴKPS',
 'ㅎㅏㄴㅈㅓㄴKSP',
 'ㅎㅏㄴㄱㅜㄱㅈㅓㄴㄹㅕㄱ KPF']

In [303]:
[t for t in no_comname]

['ㅎ', 'ㅏ', 'ㄴ', 'ㅈ', 'ㅓ', 'ㄴ', 'L', 'P', 'S']

In [364]:
tokenizer = Okt()
tokenizer.pos("한적이있다")

[('한적', 'Noun'), ('이', 'Josa'), ('있다', 'Adjective')]

In [379]:
import re

# 텍스트에 포함된 특수문자를 제거하는 함수
def remove_special_characters(text):
    # 특수문자를 제외한 모든 문자와 숫자를 남기는 정규식
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

# 예시 텍스트
text = "Hello, World! 안녕하세요? @2024#"

# 특수문자 제거
clean_text = remove_special_characters(text)
print(clean_text)


Hello World  2024


In [550]:
from jamo import h2j, j2hcj
import re

comname_origin = "한전KPS"
no_comname_origin = ["한전LPS","한전도로공사","한도로공사전","한전ICPS","한국전력 KPS","KPS한전","한국KPS","전KPS","한전KSP","한국전력 KPF",
              "한적이있다", "한적"]
no_comname_origin = [re.sub(r'[^가-힣a-zA-Z0-9]', '', c_o) for c_o in no_comname_origin]
comname_jamo = j2hcj(h2j(comname_origin))
no_comname_jamo_list = [j2hcj(h2j(choice)) for choice in no_comname_origin]

for cdx, no_comname in enumerate(no_comname_jamo_list):
    comname = ''.join(comname_jamo)
    no_comname = ''.join(no_comname)
    len_comname, len_no_comname = len(comname), len(no_comname)
    no_comname += "@"*(len_comname - len_no_comname) if len_comname > len_no_comname else ""
        
    ic(comname)
    ic(no_comname)
    partial_ratio_result = fuzz.partial_ratio(comname, no_comname)
    token_sort_ratio_result = fuzz.token_sort_ratio(comname, no_comname)
    ic(partial_ratio_result)
    ic(token_sort_ratio_result)
    if (partial_ratio_result > 60 and token_sort_ratio_result > 60):
        ic(f"{no_comname_origin[cdx]} : 검출됨")
    elif (partial_ratio_result >= 80 or token_sort_ratio_result >= 80):
        ic(f"{no_comname_origin[cdx]} : 검출됨2")
    ic("-"*100)

ic| comname: 'ㅎㅏㄴㅈㅓㄴKPS'
ic| no_comname: 'ㅎㅏㄴㅈㅓㄴLPS'
ic| partial_ratio_result: 88.88888888888889
ic| token_sort_ratio_result: 88.88888888888889
ic| f"{no_comname_origin[cdx]} : 검출됨": '한전LPS : 검출됨'
ic| "-"*100: '----------------------------------------------------------------------------------------------------'
ic| comname: 'ㅎㅏㄴㅈㅓㄴKPS'
ic| no_comname: 'ㅎㅏㄴㅈㅓㄴㄷㅗㄹㅗㄱㅗㅇㅅㅏ'
ic| partial_ratio_result: 80.0
ic| token_sort_ratio_result: 50.0
ic| f"{no_comname_origin[cdx]} : 검출됨2": '한전도로공사 : 검출됨2'
ic| "-"*100: '----------------------------------------------------------------------------------------------------'
ic| comname: 'ㅎㅏㄴㅈㅓㄴKPS'
ic| no_comname: 'ㅎㅏㄴㄷㅗㄹㅗㄱㅗㅇㅅㅏㅈㅓㄴ'
ic| partial_ratio_result: 61.53846153846154
ic| token_sort_ratio_result: 50.0
ic| "-"*100: '----------------------------------------------------------------------------------------------------'
ic| comname: 'ㅎㅏㄴㅈㅓㄴKPS'
ic| no_comname: 'ㅎㅏㄴㅈㅓㄴICPS'
ic| partial_ratio_result: 80.0
ic| token_sort_ratio_result: 84.21052631578947
ic| f"

In [529]:
comname = comname.upper()
sent = base_dict[1][0][1].upper()

# comname = re.sub(r'[^가-힣a-zA-Z0-9]', '', comname)
# sent = re.sub(r'[^가-힣a-zA-Z0-9]', '', sent)

comname_jamo = j2hcj(h2j(comname))
sent_jamo = j2hcj(h2j(sent))

print(sent)
print(sent_jamo)
print(comname_jamo)
ratio_result = fuzz.ratio(comname_jamo, sent_jamo)
partial_ratio_result = fuzz.partial_ratio(comname_jamo, sent_jamo)
token_sort_ratio_result = fuzz.token_sort_ratio(comname_jamo, sent_jamo)

print(ratio_result)
print(partial_ratio_result)
print(token_sort_ratio_result)

에너지 관련 기업을 탐색하던 중 우리나라 전력 생산 설비의 정비와 유지 보수를 담당하는 한국전력 KPS의 역할과 중요성에 대해 알게 되었습니다.
ㅇㅔㄴㅓㅈㅣ ㄱㅘㄴㄹㅕㄴ ㄱㅣㅇㅓㅂㅇㅡㄹ ㅌㅏㅁㅅㅐㄱㅎㅏㄷㅓㄴ ㅈㅜㅇ ㅇㅜㄹㅣㄴㅏㄹㅏ ㅈㅓㄴㄹㅕㄱ ㅅㅐㅇㅅㅏㄴ ㅅㅓㄹㅂㅣㅇㅢ ㅈㅓㅇㅂㅣㅇㅘ ㅇㅠㅈㅣ ㅂㅗㅅㅜㄹㅡㄹ ㄷㅏㅁㄷㅏㅇㅎㅏㄴㅡㄴ ㅎㅏㄴㄱㅜㄱㅈㅓㄴㄹㅕㄱ KPSㅇㅢ ㅇㅕㄱㅎㅏㄹㄱㅘ ㅈㅜㅇㅇㅛㅅㅓㅇㅇㅔ ㄷㅐㅎㅐ ㅇㅏㄹㄱㅔ ㄷㅚㅇㅓㅆㅅㅡㅂㄴㅣㄷㅏ.
ㅎㅏㄴㅈㅓㄴKPS
10.285714285714286
66.66666666666667
6.857142857142861


In [607]:
j2hcj(h2j("바탕으로 KPS한전의 일원이"))


'ㅂㅏㅌㅏㅇㅇㅡㄹㅗ KPSㅎㅏㄴㅈㅓㄴㅇㅢ ㅇㅣㄹㅇㅝㄴㅇㅣ'

In [608]:
src = "ㅎㅏㄴㅈㅓㄴKPS"
dst = "KPSㅎㅏㄴㅈㅓㄴ"

ratio_result = fuzz.ratio(src, dst)
partial_ratio_result = fuzz.partial_ratio(src, dst)
token_sort_ratio_result = fuzz.token_sort_ratio(src, dst)

print("Ratio 결과:", ratio_result)
print("Partial Ratio 결과:", partial_ratio_result)
print("Token Sort Ratio 결과:", token_sort_ratio_result)

Ratio 결과: 30.76923076923077
Partial Ratio 결과: 66.66666666666667
Token Sort Ratio 결과: 30.76923076923077


In [683]:
import difflib
from rapidfuzz import fuzz
from jamo import h2j, j2h, j2hcj
from unicode import join_jamos

no_comname_origin = ["한전LPS","한전도로공사","한도로공사전","한전ICPS","한국전력 KPS",
                     "KPS한전","한국KPS","전KPS","한전KSP","한국전력 KPF",
                     "한적이있다", "한적", "학전KPS"]
comname = comname.upper()
# sent = base_dict[2][3][6].upper()
sent_test = "이러한 경험을 바탕으로 ###의 일원이 되어도 반복되는 불편을 개선하는 조직원이 되도록 하겠습니다."

# comname = re.sub(r'[^가-힣a-zA-Z0-9]', '', comname)
# sent = re.sub(r'[^가-힣a-zA-Z0-9]', '', sent)

comname_jamo = j2hcj(h2j(comname))
sent_jamo = j2hcj(h2j(sent_test))

# comname_jamo = comname
# sent_jamo = sent

# print("원본 문자열:", sent_test)
# print("자모 문자열:", sent_jamo)
print("회사명:", comname_jamo)
print()
for i in no_comname_origin:
    print(f"<{i}>")
    sent = sent_test.replace("###", i)
    sent = re.sub(r'[^가-힣a-zA-Z0-9]', '', sent)
    sent_jamo = j2hcj(h2j(sent))
    print("원본 문자열:", sent)
    print("자모 문자열:", sent_jamo)
    ratio_result = fuzz.ratio(comname_jamo, sent_jamo)
    partial_ratio_result = fuzz.partial_ratio(comname_jamo, sent_jamo)
    token_sort_ratio_result = fuzz.token_sort_ratio(comname_jamo, sent_jamo)

    # print("Ratio 결과:", ratio_result)
    # print("Partial Ratio 결과:", partial_ratio_result)
    # print("Token Sort Ratio 결과:", token_sort_ratio_result)

    # if partial_ratio_result > 60:
    if partial_ratio_result >= 60:
        matcher = difflib.SequenceMatcher(None, comname_jamo, sent_jamo)
        matching_blocks = matcher.get_matching_blocks()
        first_start_b, last_start_b = 0, 0
        for bdx, block in enumerate(matching_blocks):
            # print(block)
            if block[-1] >= 1:
                start_a, start_b, size = block
                if bdx == 0:
                    first_start_b = start_b
                if bdx == len(matching_blocks)-2:
                    last_start_b = start_b + size
                print(first_start_b, last_start_b)
                print(f"회사명 '{comname_jamo[start_a:start_a+size]}'와 문장 '{sent_jamo[start_b:start_b+size]}'이 일치합니다.")
                # print(first_start_b, last_start_b)
                # print(join_jamos(comname_jamo[start_a:start_a+size]))
                # if sent_jamo[first_start_b-10:last_start_b+10]:
                if first_start_b-10 < 0:
                    first_start_b = 10
                print(join_jamos(sent_jamo[first_start_b-10:last_start_b+10]))
    else:
        print("검출 X")
    print("-"*100)

회사명: ㅎㅏㄴㅈㅓㄴKPS

<한전LPS>
원본 문자열: 이러한경험을바탕으로한전LPS의일원이되어도반복되는불편을개선하는조직원이되도록하겠습니다
자모 문자열: ㅇㅣㄹㅓㅎㅏㄴㄱㅕㅇㅎㅓㅁㅇㅡㄹㅂㅏㅌㅏㅇㅇㅡㄹㅗㅎㅏㄴㅈㅓㄴLPSㅇㅢㅇㅣㄹㅇㅝㄴㅇㅣㄷㅚㅇㅓㄷㅗㅂㅏㄴㅂㅗㄱㄷㅚㄴㅡㄴㅂㅜㄹㅍㅕㄴㅇㅡㄹㄱㅐㅅㅓㄴㅎㅏㄴㅡㄴㅈㅗㅈㅣㄱㅇㅝㄴㅇㅣㄷㅚㄷㅗㄹㅗㄱㅎㅏㄱㅔㅆㅅㅡㅂㄴㅣㄷㅏ
25 0
회사명 'ㅎㅏㄴㅈㅓㄴ'와 문장 'ㅎㅏㄴㅈㅓㄴ'이 일치합니다.

25 34
회사명 'PS'와 문장 'PS'이 일치합니다.
ㄹ바탕으로한전LPS의일원이
----------------------------------------------------------------------------------------------------
<한전도로공사>
원본 문자열: 이러한경험을바탕으로한전도로공사의일원이되어도반복되는불편을개선하는조직원이되도록하겠습니다
자모 문자열: ㅇㅣㄹㅓㅎㅏㄴㄱㅕㅇㅎㅓㅁㅇㅡㄹㅂㅏㅌㅏㅇㅇㅡㄹㅗㅎㅏㄴㅈㅓㄴㄷㅗㄹㅗㄱㅗㅇㅅㅏㅇㅢㅇㅣㄹㅇㅝㄴㅇㅣㄷㅚㅇㅓㄷㅗㅂㅏㄴㅂㅗㄱㄷㅚㄴㅡㄴㅂㅜㄹㅍㅕㄴㅇㅡㄹㄱㅐㅅㅓㄴㅎㅏㄴㅡㄴㅈㅗㅈㅣㄱㅇㅝㄴㅇㅣㄷㅚㄷㅗㄹㅗㄱㅎㅏㄱㅔㅆㅅㅡㅂㄴㅣㄷㅏ
25 31
회사명 'ㅎㅏㄴㅈㅓㄴ'와 문장 'ㅎㅏㄴㅈㅓㄴ'이 일치합니다.
ㄹ바탕으로한전도로공상
----------------------------------------------------------------------------------------------------
<한도로공사전>
원본 문자열: 이러한경험을바탕으로한도로공사전의일원이되어도반복되는불편을개선하는조직원이되도록하겠습니다
자모 문자열: ㅇㅣㄹㅓㅎㅏㄴㄱㅕㅇㅎㅓㅁㅇㅡㄹㅂㅏㅌㅏㅇㅇㅡㄹㅗㅎㅏㄴㄷㅗㄹㅗㄱㅗㅇㅅㅏㅈㅓㄴㅇㅢㅇㅣㄹㅇㅝㄴㅇㅣㄷㅚㅇㅓㄷㅗㅂㅏㄴㅂㅗㄱㄷㅚㄴㅡㄴㅂㅜㄹㅍㅕㄴㅇㅡㄹㄱㅐㅅㅓㄴㅎㅏㄴㅡㄴㅈㅗㅈㅣㄱㅇㅝㄴㅇㅣㄷㅚㄷㅗㄹㅗㄱㅎㅏㄱㅔㅆㅅㅡㅂㄴㅣㄷㅏ
검출 X
----------------------------------------------------------------------------------

---


---


---


In [856]:
chunked_sent_origin

['한전',
 '한전 KPS에서',
 'KPS에서 노력하고',
 '노력하고 이러한',
 '이러한 경험을',
 '경험을 바탕으로',
 '바탕으로 KPS한전의',
 'KPS한전의 일원이',
 '일원이 되어도',
 '되어도 반복되는',
 '반복되는 불편을',
 '불편을 개선하는',
 '개선하는 한전LPS조직원이',
 '한전LPS조직원이 되도록',
 '되도록 하겠습니다.']

In [855]:
def chunk_text(text, n=2):
    words = text.split()
    chunks = [' '.join(words[i:i+n]) for i in range(len(words)-n+1)]
    return chunks

def split_into_chunks(lst, n=2):
    chunked_list = []
    for i in range(0, len(lst), n):
        text = ""
        for jdx, j in enumerate(lst[i:i+n]):
            text += j if jdx == 0 else f" {j.split(' ')[-1]}"
            # text += f" {j.split(' ')[0]}" if jdx == 0 else j
        chunked_list.append(text)
    return chunked_list


comname_list = ["한전KPS", "한전 KPS", "한전"]

sent = "한전 KPS에서 노력하고 이러한 경험을 바탕으로 KPS한전의 일원이 되어도 반복되는 불편을 개선하는 한전LPS조직원이 되도록 하겠습니다."
chunked_sent_origin = chunk_text(sent, 2)
chunked_sent_jamo = [j2hcj(h2j(choice)) for choice in chunked_sent_origin]

chunked_sent_origin = [chunked_sent_origin[0].split(" ")[0]] + chunked_sent_origin
chunked_sent_jamo = [chunked_sent_jamo[0].split(" ")[0]] + chunked_sent_jamo

detection_list = []
for cdx, comname_origin in enumerate(comname_list):
    comname = j2hcj(h2j(comname_origin))
    for n_cdx, no_comname in enumerate(chunked_sent_jamo):
        # comname = ''.join(comname_jamo)
        # no_comname = ''.join(no_comname)
        # no_comname = no_comname.replace(comname, "#@!")
        
        ic(comname)
        ic(no_comname)
        ratio_result = fuzz.ratio(comname, no_comname)
        partial_ratio_result = fuzz.partial_ratio(comname, no_comname)
        token_sort_ratio_result = fuzz.token_sort_ratio(comname, no_comname)
        ic(ratio_result)
        ic(partial_ratio_result)
        ic(token_sort_ratio_result)
        # if (partial_ratio_result > 60 or token_sort_ratio_result > 60):
        if (partial_ratio_result > 60 and token_sort_ratio_result > 60) or \
            (partial_ratio_result >= 80 or token_sort_ratio_result >= 80):
            # if (partial_ratio_result != 100 and token_sort_ratio_result != 100):
            ic(f"{comname_origin} -> {chunked_sent_origin[n_cdx]} : 검출됨")
            detection_list.append(chunked_sent_origin[n_cdx])
        ic("-"*100)

# for detected_token in detection_list:
#     print(detected_token)
#     for pass_comname in comname_list:
#         print(pass_comname)
#     print("-"*50)
        # if pass_comname not in detected_token:
        #     print(detected_token)
            
        # print(detected_token)


result = split_into_chunks(detection_list, 2)
print(detection_list)
print(result)

ic| comname: 'ㅎㅏㄴㅈㅓㄴKPS'
ic| no_comname: 'ㅎㅏㄴㅈㅓㄴ'
ic| ratio_result: 80.0
ic| partial_ratio_result: 100.0
ic| token_sort_ratio_result: 80.0
ic| f"{comname_origin} -> {chunked_sent_origin[n_cdx]} : 검출됨": '한전KPS -> 한전 : 검출됨'
ic| "-"*100: '----------------------------------------------------------------------------------------------------'
ic| comname: 'ㅎㅏㄴㅈㅓㄴKPS'
ic| no_comname: 'ㅎㅏㄴㅈㅓㄴ KPSㅇㅔㅅㅓ'
ic| ratio_result: 78.26086956521739
ic| partial_ratio_result: 88.88888888888889
ic| token_sort_ratio_result: 52.17391304347826
ic| f"{comname_origin} -> {chunked_sent_origin[n_cdx]} : 검출됨": '한전KPS -> 한전 KPS에서 : 검출됨'
ic| "-"*100: '----------------------------------------------------------------------------------------------------'
ic| comname: 'ㅎㅏㄴㅈㅓㄴKPS'
ic| no_comname: 'KPSㅇㅔㅅㅓ ㄴㅗㄹㅕㄱㅎㅏㄱㅗ'
ic| ratio_result: 23.076923076923073
ic| partial_ratio_result: 50.0
ic| token_sort_ratio_result: 23.076923076923073
ic| "-"*100: '---------------------------------------------------------------------------------

['한전', '한전 KPS에서', '개선하는 한전LPS조직원이', '한전LPS조직원이 되도록', '한전', '한전 KPS에서', '개선하는 한전LPS조직원이', '한전LPS조직원이 되도록', '한전', '한전 KPS에서', '바탕으로 KPS한전의', 'KPS한전의 일원이', '개선하는 한전LPS조직원이', '한전LPS조직원이 되도록']
['한전 KPS에서', '개선하는 한전LPS조직원이 되도록', '한전 KPS에서', '개선하는 한전LPS조직원이 되도록', '한전 KPS에서', '바탕으로 KPS한전의 일원이', '개선하는 한전LPS조직원이 되도록']


---

In [871]:
chunked_sent_jamo

['ㅎㅏㄴㅈㅓㄴ',
 'KPSㅇㅔㅅㅓ',
 'ㄴㅗㄹㅕㄱㅎㅏㄱㅗ',
 'ㅇㅣㄹㅓㅎㅏㄴ',
 'ㄱㅕㅇㅎㅓㅁㅇㅡㄹ',
 'ㅂㅏㅌㅏㅇㅇㅡㄹㅗ',
 'KPSㅎㅏㄴㅈㅓㄴㅇㅢ',
 'ㅇㅣㄹㅇㅝㄴㅇㅣ',
 'ㄷㅚㅇㅓㄷㅗ',
 'ㅂㅏㄴㅂㅗㄱㄷㅚㄴㅡㄴ',
 'ㅂㅜㄹㅍㅕㄴㅇㅡㄹ',
 'ㄱㅐㅅㅓㄴㅎㅏㄴㅡㄴ',
 'ㅎㅏㄴㅈㅓㄴLPSㅈㅗㅈㅣㄱㅇㅝㄴㅇㅣ',
 'ㄷㅚㄷㅗㄹㅗㄱ',
 'ㅎㅏㄱㅔㅆㅅㅡㅂㄴㅣㄷㅏ.']

In [1234]:
not any([i == 100 for i in [10,10,100]])

False

In [1363]:
"가나다라 마바사".index(" ")

4

In [7]:
with open("./jasa_dict_0910_test.json", "r") as f:
    base_dict = json.load(f)
def convert_keys_to_int(d):
    if isinstance(d, dict):
        new_dict = {}
        for key, value in d.items():
            # 문자열 키를 int로 변환 (가능할 경우에만)
            new_key = int(key) if key.isdigit() else key
            new_dict[new_key] = convert_keys_to_int(value)
        return new_dict
    elif isinstance(d, list):
        return [convert_keys_to_int(item) for item in d]
    else:
        return d
base_dict = convert_keys_to_int(base_dict)
base_dict

{0: {0: {0: '한전LPS에 들어가고 싶습니다.',
   1: '뉴스를 통해 ‘2030, 2050 탄소 중립’을 접하게 되었고 단순하게 생각했던 자연재해나 이상기온의 원인이 일회용품과 화석연료에서 배출되는 탄소 때문에 일어나고 있다는 것을 알게 되었습니다.',
   2: '배출 온실가스의 대부분이 에너지 소비과정에서 발생하고 있으므로 에너지 분야는 탄소 중립 실현의 핵심 분야라고 생각합니다.',
   3: '그 중 한전kps는 안전한 전력공급과 함께 전기차충전사업, 태양광발전사업 등의 에너지신사업을 주도하며 친환경적인 움직임을 보이고 있습니다.',
   4: '0000과학기술원에서 연구본부지원실에 소속해 업무추진비 지급신청과 지출 관리의 회계 업무를 담당하였습니다.',
   5: '이때 문제해결역량을 기를 수 있었습니다.',
   6: '증빙서류 누락이나 잘못 기재되는 등 재요청을 하는 건수를 개선안을 마련하여 한 달에 8회에서 0건으로 만들었습니다.',
   7: '이때 통신비 납부 지연으로 인해 발생한 수수료 300원을 납부 방법변경으로 미발생하게 하였습니다.'},
  1: {0: '우리나라는 해외에 비해 친환경 에너지를 사용하기에 기술적, 지형적문제가 있습니다.',
   1: '이를 극복하기위한 방면으로는 그린 뉴딜 정책이 가장 적합하다고 생각하며 한국에 맞는 운영을 할 때 더욱 효율적인 운영이 필요하고 이를 위해서 디지털화가 중심이 되어야 한다고 생각합니다.',
   2: '디지털 뉴딜을 통해 공공일자리를 넘어 숙련된 질높은 일자리를 구축하는 등 구조적 변화를 해야합니다.',
   3: '이를 시행할 때 있어서 무엇보다 안전한 전력수급이 중요합니다.',
   4: '이를 위해서는 전력과 디지털 분야 두 분야에 특화된 전력관련한 디지털 지식을 가지고 숙련된 인재를 양성하고 대기시키는 게 중요하고 기존 설비에 대해서는 주기적인 검사를 통해 예방을 해야합니다.',
   5: '따라서 향후에 디지털 뉴딜을 중심으로 지속가능한 미래를 만들도록 미래사업에 

In [280]:
s ="ㅎㅏㄴㅈㅓㄴKPS"
d = "ㅎㅏㄴ ㅈㅓㄴ@@"
ratio_result = fuzz.ratio(s, d)
partial_ratio_result = fuzz.partial_ratio(s, d)
token_sort_ratio_result = fuzz.token_sort_ratio(s, d)
print(ratio_result)
print(partial_ratio_result)
print(token_sort_ratio_result)

66.66666666666667
80.0
33.333333333333336


In [125]:
from rapidfuzz import fuzz
from jamo import h2j, j2h, j2hcj
from unicode import join_jamos
from itertools import combinations, permutations
import copy
from konlpy.tag import Okt
from tqdm import tqdm
import json

with open("./jasa_dict_0910_test.json", "r") as f:
    base_dict = json.load(f)
def convert_keys_to_int(d):
    if isinstance(d, dict):
        new_dict = {}
        for key, value in d.items():
            # 문자열 키를 int로 변환 (가능할 경우에만)
            new_key = int(key) if key.isdigit() else key
            new_dict[new_key] = convert_keys_to_int(value)
        return new_dict
    elif isinstance(d, list):
        return [convert_keys_to_int(item) for item in d]
    else:
        return d
base_dict = convert_keys_to_int(base_dict)

# def check_window(n_cdx, chunked_sent_jamo, splited_sent, comname_origin, rs, prs, tsrs, pass_list, sent, word_ind_list, key_word_list, detect_equal_list):
def check_window(n_cdx, chunked_sent_jamo, splited_sent, comname_origin, rs, prs, tsrs, pass_list, mini_dict):
    start_cdx = n_cdx
    end_cdx = len(chunked_sent_jamo) if n_cdx+2 > len(chunked_sent_jamo)-1 else n_cdx+2
    splited_sent_origin = splited_sent[start_cdx: end_cdx]
    detect_equal = False
    
    for sdx, splited_token in enumerate(splited_sent_origin):
        for pass_token in pass_list:
            if pass_token in splited_token:
                splited_sent_origin[sdx] = "```"            
    
    new_no_comname_origin = " ".join(splited_sent_origin).upper()
    # for pass_token in pass_list:
    #     new_no_comname_origin = new_no_comname_origin.replace(pass_token, "```")
    
    comname = j2hcj(h2j(comname_origin)).upper()
    len_comname = len(comname)
    best_ratio = rs + prs + tsrs
    split_num = None
    best_token = ""
    for idx, j in enumerate(new_no_comname_origin):
        txt = ""
        tt = new_no_comname_origin[idx:]
        for j in tt:
            txt += j
            new_comname = comname[:]
            new_comname += j if j == " " else ""
            new_jamo = j2hcj(h2j(txt))
            # len_new_comname = len(new_comname)
            len_new_jamo = len(new_jamo)
            new_comname += "@"*(len_new_jamo - len_comname) if len_new_jamo > len_comname else ""
            new_jamo += "@"*(len_comname - len_new_jamo) if len_comname > len_new_jamo else ""
            
            ratio_result = fuzz.ratio(new_comname, new_jamo)
            partial_ratio_result = fuzz.partial_ratio(new_comname, new_jamo)
            token_sort_ratio_result = fuzz.token_sort_ratio(new_comname, new_jamo)
            
            if (ratio_result + partial_ratio_result + token_sort_ratio_result >= best_ratio):
                # print(new_jamo)
                # print(ratio_result, partial_ratio_result, token_sort_ratio_result)
                # print(rs, prs, tsrs)
                # if not all([i == 100 for i in [ratio_result, partial_ratio_result, token_sort_ratio_result]]):
                if len(comname_origin) == 2:
                    # if not any([i > 80 for i in [ratio_result, partial_ratio_result, token_sort_ratio_result]]):
                    #     continue
                    ratio_list = [ratio_result, partial_ratio_result, token_sort_ratio_result]
                    if all(x > 60 for x in ratio_list):  # 1. 모두 60보다 커야 함
                        if any(x > 80 for x in ratio_list):  # 2. 셋 중에 최소한 하나는 80을 넘어야 함
                            pass
                        else:
                            continue
                    else:
                        continue
                if all([i == 100 for i in [ratio_result, partial_ratio_result, token_sort_ratio_result]]):
                    print("100!!!!!!!!!!!!!!!!!!!!!!!!!!!!", new_comname, new_jamo)
                    detect_equal = True
                    # continue
                    # break
                    # return detected_dict
                # print(new_comname)
                # print(new_jamo)
                best_ratio = ratio_result + partial_ratio_result + token_sort_ratio_result
                best_token = new_jamo.replace("@","")
                if " " in new_no_comname_origin:
                    if idx >= new_no_comname_origin.index(" "):
                        split_num = 1
                    else:
                        split_num = 0
                # print("#################### :", new_comname, "//", new_jamo, [ratio_result, partial_ratio_result, token_sort_ratio_result])
                
                # if any([i == 100 for i in [ratio_result, partial_ratio_result, token_sort_ratio_result]]):
                #     print("#################### :", new_comname)
                    # print("!@#!@#!@#!@#", best_token)
    # else:
    #     print(100)          
    if best_token:
        final_token_split = join_jamos(best_token.strip()).split(" ")
        if split_num is not None:
            if len(final_token_split) == 1:
                if split_num == 0:
                    final_token_split = final_token_split + ["!@#"]
                if split_num == 1:
                    final_token_split = ["!@#"] + final_token_split

        detected_token = []
        # print(final_token_split)
        # print(splited_sent_origin)
        for f_token, s_token in zip(final_token_split, splited_sent_origin):
            # if f_token in s_token:
            if f_token != "!@#":
                detected_token.append(s_token)
        
        final_detected_token = " ".join(detected_token)
        # if final_detected_token not in detected_dict[comname_origin]:
        #     detected_dict[comname_origin].append(final_detected_token)
        # detected_dict['word_ind'] = detected_dict['sent'].index(final_detected_token)
        # detected_dict['key_word'] = final_detected_token
        # if detect_equal:
        #     check_pos_pass = check_pos(comname_origin, final_detected_token)
        # else:
        #    check_pos_pass = True 
        
        # if check_pos_pass:
        # if final_detected_token not in detected_dict['key_word']:
        # detected_dict['word_ind'].append(detected_dict['sent'].index(final_detected_token))
        # detected_dict['key_word'].append(final_detected_token)
        # detected_dict['detect_equal'].append(detect_equal)
        # word_ind_list.append(sent.index(final_detected_token))
        # key_word_list.append(final_detected_token)
        # detect_equal_list.append(detect_equal)
        mini_dict['word_ind'].append(mini_dict['sent'].index(final_detected_token))
        mini_dict['key_word'].append(final_detected_token)
        mini_dict['detect_equal'].append(detect_equal)
            
        # print(">>>>>", final_detected_token)
        # print(f"<{comname_origin}>")
        # print(detected_dict)
        # print(sent)
        # print(word_ind_list)
        # print(key_word_list)
        # print(detect_equal_list)
        # print("="*150)
    # return word_ind_list, key_word_list, detect_equal_list
        # print(mini_dict)
    return mini_dict

In [126]:
# def check_pos(comname, detected_token):
#     tokenizer = Okt()
#     comname = comname.upper()
#     detected_token = detected_token.upper()
#     window_pass_pos = ['Punctuation', 'Josa', 'Verb', 'Adjective', 'Noun']
#     window_pass_pos = list(permutations(window_pass_pos,1)) + \
#                         list(permutations(window_pass_pos,2))
#     space = True if " " in comname else False                    
#     detected_com = detected_token.replace(comname, "꽭뛼쨿쀎")
#     chk_com = ('꽭뛼쨿쀎', 'Noun')
#     pos_tag_list = tokenizer.pos(detected_com)
#     print(pos_tag_list)
#     com_idx = pos_tag_list.index(chk_com)
#     start_idx, end_idx = (0,2) if com_idx == 0 else (com_idx-1, com_idx+2)
#     window_pos_tag_list = pos_tag_list[start_idx: end_idx] 
#     print(pos_tag_list)
#     print(window_pos_tag_list)
#     window_pos_tag_list = [i for i in window_pos_tag_list if i != chk_com]
#     print(window_pos_tag_list)
#     if len(window_pos_tag_list) == 0:
#         # pop_idx = detected_dict_test[k].index(v_v)
#         # detected_dict_test[k].pop(pop_idx)
#         return False
#     else:
#         chk_window_pos = tuple([i[-1] for i in window_pos_tag_list])
#         if chk_window_pos in window_pass_pos:
#             # pop_idx = detected_dict_test[k].index(v_v)
#             # detected_dict_test[k].pop(pop_idx)
#             # print("잘못 없음!")
#             return False
#     return True

# def check_pos(comname, word_ind_list, key_word_list, detect_equal_list, mini_dict):
#     print(comname, word_ind_list, key_word_list, detect_equal_list, mini_dict)
#     tokenizer = Okt()
#     comname = comname.upper()
#     window_pass_pos = ['Punctuation', 'Josa', 'Verb', 'Adjective', 'Noun']
#     window_pass_pos = list(permutations(window_pass_pos,1)) + \
#                         list(permutations(window_pass_pos,2))
#     space = True if " " in comname else False    
    
#     for wi, kw, de in zip(word_ind_list, key_word_list, detect_equal_list):
#         if de:
#             print(wi, kw, de)
#             detected_com = kw.upper().replace(comname, "꽭뛼쨿쀎")
#             chk_com = ('꽭뛼쨿쀎', 'Noun')
#             pos_tag_list = tokenizer.pos(detected_com)
#             com_idx = pos_tag_list.index(chk_com)
#             start_idx, end_idx = (0,2) if com_idx == 0 else (com_idx-1, com_idx+2)
#             window_pos_tag_list = pos_tag_list[start_idx: end_idx] 
#             print(pos_tag_list)
#             print(window_pos_tag_list)
#             window_pos_tag_list = [i for i in window_pos_tag_list if i != chk_com]
#             print(window_pos_tag_list)
#             if len(window_pos_tag_list) == 0:
#                 # pop_idx = detected_dict_test[k].index(v_v)
#                 # detected_dict_test[k].pop(pop_idx)
#                 print("문제없음!!!!!!")
#                 # return False
#             else:
#                 chk_window_pos = tuple([i[-1] for i in window_pos_tag_list])
#                 if chk_window_pos in window_pass_pos:
#                     # pop_idx = detected_dict_test[k].index(v_v)
#                     # detected_dict_test[k].pop(pop_idx)
#                     # print("잘못 없음!")
#                     print("문제없음!!!!!!")
#                     # return False
#     # return True

def check_pos(comname, mini_dict):
    print(mini_dict)
    # tokenizer = Okt()
    # comname = comname.upper()
    # detected_token = detected_token.upper()
    # window_pass_pos = ['Punctuation', 'Josa', 'Verb', 'Adjective', 'Noun']
    # window_pass_pos = list(permutations(window_pass_pos,1)) + \
    #                     list(permutations(window_pass_pos,2))
    # space = True if " " in comname else False                    
    # detected_com = detected_token.replace(comname, "꽭뛼쨿쀎")
    # chk_com = ('꽭뛼쨿쀎', 'Noun')
    # pos_tag_list = tokenizer.pos(detected_com)
    # print(pos_tag_list)
    # com_idx = pos_tag_list.index(chk_com)
    # start_idx, end_idx = (0,2) if com_idx == 0 else (com_idx-1, com_idx+2)
    # window_pos_tag_list = pos_tag_list[start_idx: end_idx] 
    # print(pos_tag_list)
    # print(window_pos_tag_list)
    # window_pos_tag_list = [i for i in window_pos_tag_list if i != chk_com]
    # print(window_pos_tag_list)
    # if len(window_pos_tag_list) == 0:
    #     # pop_idx = detected_dict_test[k].index(v_v)
    #     # detected_dict_test[k].pop(pop_idx)
    #     return False
    # else:
    #     chk_window_pos = tuple([i[-1] for i in window_pos_tag_list])
    #     if chk_window_pos in window_pass_pos:
    #         # pop_idx = detected_dict_test[k].index(v_v)
    #         # detected_dict_test[k].pop(pop_idx)
    #         # print("잘못 없음!")
    #         return False
    # return True

In [47]:
comname_list = ["한전KPS", "한전 KPS", "한전"]
sent = "한전 KPS에서 노력하고 이러한 한적KPS과 그냥 한전KPS저럼 고냥 한전 KPㄴT처럼 그냥 한전KPST더럼 이러한 남태현 한전ㄴ인이 될래요. \
    경험을 바탕하ㄴ전KPS으로 다음 KPS한전의 일원이 되어도 한전PS 그리고 한국전력KPF 반복되는 불편을 개선하는 한남자가 한전LPS조직원이 되도록 하겠습니다 한전도로공사 최고.\
        멋진한전직원이 될래요"
splited_sent = sent.split()
chunked_sent_jamo = [j2hcj(h2j(choice)) for choice in splited_sent]

detected_dict = dict()

for cdx, comname_origin in enumerate(comname_list):
    detected_dict[comname_origin] = []
    comname = j2hcj(h2j(comname_origin))
    len_comname = len(comname)
    for n_cdx, no_comname in enumerate(chunked_sent_jamo):
        len_no_comname = len(no_comname)
        no_comname += "@"*(len_comname - len_no_comname) if len_comname > len_no_comname else ""
        # print(comname)
        # print(no_comname)
        ratio_result = fuzz.ratio(comname, no_comname)
        partial_ratio_result = fuzz.partial_ratio(comname, no_comname)
        token_sort_ratio_result = fuzz.token_sort_ratio(comname, no_comname)
        # if (partial_ratio_result > 60 and token_sort_ratio_result > 60) or \
        #     (partial_ratio_result >= 70 or token_sort_ratio_result >= 70):
            # if all([i == 100 for i in [ratio_result,partial_ratio_result,token_sort_ratio_result]]):
                # continue
        if any([i > 60 for i in [ratio_result,partial_ratio_result,token_sort_ratio_result]]):
            print(f"{comname_origin} -> {splited_sent[n_cdx]} : 검출됨")
            print(ratio_result)
            print(partial_ratio_result)
            print(token_sort_ratio_result)
            check_window(n_cdx, chunked_sent_jamo, splited_sent, comname_origin, 
                         ratio_result, partial_ratio_result, token_sort_ratio_result, 
                         detected_dict)
            print("-"*100)

한전KPS -> 한전 : 검출됨
66.66666666666667
80.0
66.66666666666667


TypeError: check_window() missing 4 required positional arguments: 'sent', 'word_ind_list', 'key_word_list', and 'detect_equal_list'

In [147]:
# from rapidfuzz import fuzz
# from jamo import h2j, j2h, j2hcj
# from unicode import join_jamos

def main(comname_list, pass_list, base_dict):
    output_dict = {}
    for user, rows in tqdm(base_dict.items()):
        if user not in output_dict:
            output_dict[user] = {}
        for col, docs in rows.items():
            if col not in output_dict[user]:
                output_dict[user][col] = {}
            for sent_ind, sent in docs.items():
                mini_dict = {'sent_ind': sent_ind + 1, 'sent': sent, 'word_ind' : [], 'key_word' : [], 'detect_equal' : []}
                splited_sent = sent.split()
                chunked_sent_jamo = [j2hcj(h2j(choice)) for choice in splited_sent]
                for cdx, comname_origin in enumerate(comname_list):
                    word_ind_list, key_word_list, detect_equal_list = [], [], []
                    comname = j2hcj(h2j(comname_origin))
                    len_comname = len(comname)
                    for n_cdx, no_comname in enumerate(chunked_sent_jamo):
                        len_no_comname = len(no_comname)
                        no_comname += "@"*(len_comname - len_no_comname) if len_comname > len_no_comname else ""
                        ratio_result = fuzz.ratio(comname, no_comname)
                        partial_ratio_result = fuzz.partial_ratio(comname, no_comname)
                        token_sort_ratio_result = fuzz.token_sort_ratio(comname, no_comname)
                        if any([i > 60 for i in [ratio_result,partial_ratio_result,token_sort_ratio_result]]):
                            # word_ind_list, key_word_list, detect_equal_list = \
                            #     check_window(n_cdx, chunked_sent_jamo, splited_sent, comname_origin, 
                            #                     ratio_result, partial_ratio_result, token_sort_ratio_result, 
                            #                     pass_list, sent, word_ind_list, key_word_list, detect_equal_list)
                            mini_dict = \
                                check_window(n_cdx, chunked_sent_jamo, splited_sent, comname_origin, 
                                                ratio_result, partial_ratio_result, token_sort_ratio_result, 
                                                pass_list, mini_dict)
####################################################    
                # if any(detect_equal_list):
                #     check_pos(comname_origin, word_ind_list, key_word_list, detect_equal_list, mini_dict)
                #     print("-"*200)
                #     print("-"*200)
                # else:
                #     mini_dict['word_ind'].extend(word_ind_list)
                #     mini_dict['key_word'].extend(key_word_list)
                if any(mini_dict['detect_equal']):
                    check_pos(comname_list, mini_dict)
                    print("-"*200)
                    print("-"*200)
                # else:
                #     mini_dict['word_ind'].extend(word_ind_list)
                #     mini_dict['key_word'].extend(key_word_list)
####################################################            
    return  

def check_pos(comname_list, mini_dict):
    tokenizer = Okt()
    window_pass_pos = ['Punctuation', 'Josa', 'Verb', 'Adjective', 'Noun']
    window_pass_pos = list(permutations(window_pass_pos,1)) + \
                        list(permutations(window_pass_pos,2))
    word_ind_list = mini_dict['word_ind'][:]
    key_word_list = mini_dict['key_word'][:]
    detect_equal_list = mini_dict['detect_equal'][:]
    print(word_ind_list)
    print(key_word_list)
    print(detect_equal_list)
    pop_idx_list = []
    for ddx, de in enumerate(detect_equal_list):
        if de:
            equal_key_word = key_word_list[ddx]
            equal_key_word = equal_key_word.upper()
            for comname in comname_list:
                comname = comname.upper()
                if comname in equal_key_word: 
                    space = True if " " in comname else False
                    print(comname, space, equal_key_word)
                    detected_com = equal_key_word.replace(comname, "꽭뛼쨿쀎")
                    chk_com = ('꽭뛼쨿쀎', 'Noun')
                    pos_tag_list = tokenizer.pos(detected_com)
                    print(pos_tag_list)
                    com_idx = pos_tag_list.index(chk_com)
                    start_idx, end_idx = (0,2) if com_idx == 0 else (com_idx-1, com_idx+2)
                    window_pos_tag_list = pos_tag_list[start_idx: end_idx] 
                    print(window_pos_tag_list)
                    window_pos_tag_list = [i for i in window_pos_tag_list if i != chk_com]
                    print(window_pos_tag_list)
                    if len(window_pos_tag_list) == 0:
                        # pop_idx = detected_dict_test[k].index(v_v)
                        # detected_dict_test[k].pop(pop_idx)
                        # return False
                        pop_idx_list.append(ddx)
                    else:
                        chk_window_pos = tuple([i[-1] for i in window_pos_tag_list])
                        if chk_window_pos in window_pass_pos:
                            # pop_idx = detected_dict_test[k].index(v_v)
                            # detected_dict_test[k].pop(pop_idx)
                            # print("잘못 없음!")
                            # return False
                            pop_idx_list.append(ddx)
                            
    print(pop_idx_list)
    pop_word_ind_list = []
    for pop_idx in pop_idx_list:
        pop_word_ind = word_ind_list[pop_idx]
        for wdx, wi in enumerate(word_ind_list):
            if wi == pop_word_ind:
                pop_word_ind_list.append(wdx)
                print(wdx, wi)
                
    
    for pdx in pop_word_ind_list[::-1]:
        if mini_dict['word_ind'] and mini_dict['key_word']:
            mini_dict['word_ind'].pop(pdx)
            mini_dict['key_word'].pop(pdx)
        
    del mini_dict['detect_equal']
    print(mini_dict)
    
main(['한전KPS', '한전 KPS', '한전'], ['안전', '한번'], base_dict)

##### word_ind, key_word 중복 제거
##### word_ind 가 동일할 때는 key_word가 긴거로 선택
##### word_ind순으로 key_word정렬

 29%|█████████████████████████▍                                                               | 2/7 [00:00<00:00, 15.60it/s]

100!!!!!!!!!!!!!!!!!!!!!!!!!!!! ㅎㅏㄴㅈㅓㄴ ㅎㅏㄴㅈㅓㄴ
[0, 0, 0]
['한전LPS에', '한전LPS에', '한전LPS에']
[False, False, True]
한전 False 한전LPS에
[('꽭뛼쨿쀎', 'Noun'), ('LPS', 'Alpha'), ('에', 'Josa')]
[('꽭뛼쨿쀎', 'Noun'), ('LPS', 'Alpha')]
[('LPS', 'Alpha')]
[]
{'sent_ind': 1, 'sent': '한전LPS에 들어가고 싶습니다.', 'word_ind': [0, 0, 0], 'key_word': ['한전LPS에', '한전LPS에', '한전LPS에']}
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
100!!!!!!!!!!!!!!!!!!!!!!!!!!!! ㅎㅏㄴㅈㅓㄴKPS ㅎㅏㄴㅈㅓㄴKPS
100!!!!!!!!!!!!!!!!!!!!!!!!!!!! ㅎㅏㄴㅈㅓㄴ ㅎㅏㄴㅈㅓㄴ
[4, 4, 4]
['한전kps는', '한전kps는', '한전kps는']
[True, False, True]
한전KPS False 한전KPS는
[('꽭뛼쨿쀎', 'Noun'), ('는', 'Verb')]
[('꽭뛼쨿쀎', 'Noun'), ('는', 'Verb')]
[('는'

 57%|██████████████████████████████████████████████████▊                                      | 4/7 [00:00<00:00, 13.41it/s]

[113, 113, 113]
['한전 KPS에서도', '한전 KPS에서도', '한전']
[False, True, True]
한전 KPS True 한전 KPS에서도
[('꽭뛼쨿쀎', 'Noun'), ('에서도', 'Josa')]
[('꽭뛼쨿쀎', 'Noun'), ('에서도', 'Josa')]
[('에서도', 'Josa')]
한전 False 한전 KPS에서도
[('꽭뛼쨿쀎', 'Noun'), ('KPS', 'Alpha'), ('에서도', 'Josa')]
[('꽭뛼쨿쀎', 'Noun'), ('KPS', 'Alpha')]
[('KPS', 'Alpha')]
한전 False 한전
[('꽭뛼쨿쀎', 'Noun')]
[('꽭뛼쨿쀎', 'Noun')]
[]
[1, 2]
0 113
1 113
2 113
0 113
1 113
2 113
{'sent_ind': 4, 'sent': '동아리 내에서의 규칙과 원칙을 지키면서, 회원들과의 신뢰를 형성하고, 동아리 활동을 원활하게 진행할 수 있었습니다 이러한 경험을 통해, 저는 신뢰 형성과 원칙 준수의 중요성을 깨달았으며, 이를 바탕으로 한전 KPS에서도 원칙을 준수하고 신뢰를 형성하는 조직 구성원이 되고자 합니다.', 'word_ind': [], 'key_word': []}
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

100%|█████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 14.36it/s]

100!!!!!!!!!!!!!!!!!!!!!!!!!!!! ㅎㅏㄴㅈㅓㄴ ㅎㅏㄴㅈㅓㄴ
[134, 134, 134]
['한전', '한전 채용공고를', '한전']
[False, False, True]
한전 False 한전
[('꽭뛼쨿쀎', 'Noun')]
[('꽭뛼쨿쀎', 'Noun')]
[]
[2]
0 134
1 134
2 134
{'sent_ind': 2, 'sent': '최근 전 직장 동료에게 발전소 배관 용접 TIG+CO2 제안이 들어와 고민 중에 다른 동료와 얘기 중 공기업도 용접사를 뽑는다는 얘기를 들었고 저 또한 일용직이 아닌 가족에게 걱정을 주지 않고 정착할 수 있는 공기업을 원하게 되어 찾아보다 한전 채용공고를 보게 되어 지원할 수 있게 되었습니다.', 'word_ind': [], 'key_word': []}
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
100!!!!!!!!!!!!!!!!!!!!!!!!!!!! ㅎㅏㄴㅈㅓㄴ ㅎㅏㄴㅈㅓㄴ
[13, 13, 13]
['한전', '한전 발전소의', '한전']
[False, False, True]
한전 False 한전
[('꽭뛼쨿쀎', 'Noun')]
[('꽭뛼쨿쀎', 'Noun')]
[]
[2]
0 13
1 13
2 13
{'sent_ind': 3,

---

In [1508]:
list(combinations(['한전', ' ', 'KPS', '저럼'], 3))

[('한전', ' ', 'KPS'),
 ('한전', ' ', '저럼'),
 ('한전', 'KPS', '저럼'),
 (' ', 'KPS', '저럼')]

In [397]:
from itertools import combinations, permutations
import copy
detected_dict_copy = copy.deepcopy(detected_dict)
detected_dict_test = copy.deepcopy(detected_dict)
tokenizer = Okt()
window_pass_pos = ['Punctuation', 'Josa', 'Verb', 'Adjective', 'Noun']
window_pass_pos = list(permutations(window_pass_pos,1)) + \
                    list(permutations(window_pass_pos,2))
for k, v in detected_dict_copy.items():
    print(k, v)
    print()
    space = True if " " in k else False
    for v_v in v:
        if k in v_v:  
            print(v_v)
            detected_com = v_v.replace(k, "기업명")
            chk_com = ('기업명', 'Noun')
            pos_tag_list = tokenizer.pos(detected_com)
            com_idx = pos_tag_list.index(chk_com)
            start_idx, end_idx = (0,2) if com_idx == 0 else (com_idx-1, com_idx+2)
            window_pos_tag_list = pos_tag_list[start_idx: end_idx]
            print(pos_tag_list)
            print(window_pos_tag_list)
            window_pos_tag_list = [i for i in window_pos_tag_list if i != chk_com]
            print(window_pos_tag_list)
            if len(window_pos_tag_list) == 0:
                pop_idx = detected_dict_test[k].index(v_v)
                detected_dict_test[k].pop(pop_idx)
            else:
                chk_window_pos = tuple([i[-1] for i in window_pos_tag_list])
                if chk_window_pos in window_pass_pos:
                    pop_idx = detected_dict_test[k].index(v_v)
                    detected_dict_test[k].pop(pop_idx)
                    print("잘못 없음!")
            
        #     pos_list = [i[0] for i in tokenizer.pos(v_v)]
        #     if space:
        #         pos_list_len = [len("".join(pos_list[:idx+1])) for idx, i in enumerate(pos_list)]
        #         for pdx, p_l in enumerate(pos_list_len):
        #             if p_l == v_v.index(" "):
        #                 pos_list.insert(pdx+1, " ")
        #     print("***", pos_list)
        #     print("***", tokenizer.pos(v_v))
        #     for n in range(1, len(pos_list)+1):
        #         if k in ["".join(i) for i in list(combinations(pos_list, n))]:
        #             print(k)
        #             pop_index = detected_dict[k].index(v_v)
        #             detected_dict[k].pop(pop_index)

            print("-"*50)   
    print()

한전KPS ['한전 KPS에서', '한적KPS과', '한전KPS저럼', '한전 KPㄴT처럼', '한전KPST더럼', '한전ㄴ인이', '바탕하ㄴ전KPS으로', 'KPS한전의', '한전PS', '한국전력KPF', '한전LPS조직원이', '한전도로공사', '멋진한전직원이']

한전KPS저럼
[('기업명', 'Noun'), ('저럼', 'Verb')]
[('기업명', 'Noun'), ('저럼', 'Verb')]
[('저럼', 'Verb')]
잘못 없음!
--------------------------------------------------
한전KPST더럼
[('기업명', 'Noun'), ('T', 'Alpha'), ('더럼', 'Noun')]
[('기업명', 'Noun'), ('T', 'Alpha')]
[('T', 'Alpha')]
--------------------------------------------------

한전 KPS ['한전 KPS에서', '한적KPS과', '한전KPS저럼', '한전 KPㄴT처럼', '한전KPST더럼', '한전ㄴ인이', '바탕하ㄴ전KPS으로', 'KPS한전의', '한전PS', '한국전력KPF', '한전LPS조직원이', '한전도로공사']

한전 KPS에서
[('기업명', 'Noun'), ('에서', 'Josa')]
[('기업명', 'Noun'), ('에서', 'Josa')]
[('에서', 'Josa')]
잘못 없음!
--------------------------------------------------

한전 ['한전', '한적KPS과', '한전KPS저럼', '한전KPST더럼', '한전ㄴ인이', '바탕하ㄴ전KPS으로', 'KPS한전의', '한전PS', '한전LPS조직원이', '한전도로공사', '멋진한전직원이']

한전
[('기업명', 'Noun')]
[('기업명', 'Noun')]
[]
--------------------------------------------------
한전KPS저럼
[('기업명', 'Noun'), ('

In [395]:
detected_dict_test

{'한전KPS': ['한전 KPS에서',
  '한적KPS과',
  '한전 KPㄴT처럼',
  '한전KPST더럼',
  '한전ㄴ인이',
  '바탕하ㄴ전KPS으로',
  'KPS한전의',
  '한전PS',
  '한국전력KPF',
  '한전LPS조직원이',
  '한전도로공사',
  '멋진한전직원이'],
 '한전 KPS': ['한적KPS과',
  '한전KPS저럼',
  '한전 KPㄴT처럼',
  '한전KPST더럼',
  '한전ㄴ인이',
  '바탕하ㄴ전KPS으로',
  'KPS한전의',
  '한전PS',
  '한국전력KPF',
  '한전LPS조직원이',
  '한전도로공사'],
 '한전': ['한적KPS과',
  '한전KPS저럼',
  '한전KPST더럼',
  '한전ㄴ인이',
  '바탕하ㄴ전KPS으로',
  'KPS한전의',
  '한전PS',
  '한전LPS조직원이']}

In [415]:
tokenizer.pos("기업명는")

[('기업', 'Noun'), ('명', 'Suffix'), ('는', 'Josa')]

In [382]:
## 기업명 일치로 검출 안되게
list(permutations(['Punctuation', 'Josa', 'Verb', 'Adjective'],1)) + list(permutations(['Punctuation', 'Josa', 'Verb', 'Adjective'],2))

[('Punctuation',),
 ('Josa',),
 ('Verb',),
 ('Adjective',),
 ('Punctuation', 'Josa'),
 ('Punctuation', 'Verb'),
 ('Punctuation', 'Adjective'),
 ('Josa', 'Punctuation'),
 ('Josa', 'Verb'),
 ('Josa', 'Adjective'),
 ('Verb', 'Punctuation'),
 ('Verb', 'Josa'),
 ('Verb', 'Adjective'),
 ('Adjective', 'Punctuation'),
 ('Adjective', 'Josa'),
 ('Adjective', 'Verb')]

In [1518]:
detected_dict

{'한전KPS': ['한전 KPS에서',
  '한적KPS과',
  '한전 KPㄴT처럼',
  '한전KPST더럼',
  '한전ㄴ',
  '바탕하ㄴ전KPS으로',
  'KPS한전의',
  '한전PS',
  '한국전력KPF',
  '한전LPS조직원이',
  '한전도로공사.'],
 '한전 KPS': ['한적KPS과',
  '한전KPS저럼',
  '한전 KPㄴT처럼',
  '한전KPST더럼',
  '한전ㄴ',
  '바탕하ㄴ전KPS으로',
  'KPS한전의',
  '한전PS',
  '한국전력KPF',
  '한전LPS조직원이',
  '한전도로공사.'],
 '한전': ['한적KPS과', '바탕하ㄴ전KPS으로']}

In [53]:
corpus = []
for k_1, v_1 in base_dict.items():
    for k_2, v_2 in base_dict[k_1].items():
        for k_3, v_3 in base_dict[k_1][k_2].items():
            corpus += [v_3]
            
corpus

['한전LPS에 들어가고 싶습니다.',
 '뉴스를 통해 ‘2030, 2050 탄소 중립’을 접하게 되었고 단순하게 생각했던 자연재해나 이상기온의 원인이 일회용품과 화석연료에서 배출되는 탄소 때문에 일어나고 있다는 것을 알게 되었습니다.',
 '배출 온실가스의 대부분이 에너지 소비과정에서 발생하고 있으므로 에너지 분야는 탄소 중립 실현의 핵심 분야라고 생각합니다.',
 '그 중 한전kps는 안전한 전력공급과 함께 전기차충전사업, 태양광발전사업 등의 에너지신사업을 주도하며 친환경적인 움직임을 보이고 있습니다.',
 '0000과학기술원에서 연구본부지원실에 소속해 업무추진비 지급신청과 지출 관리의 회계 업무를 담당하였습니다.',
 '이때 문제해결역량을 기를 수 있었습니다.',
 '증빙서류 누락이나 잘못 기재되는 등 재요청을 하는 건수를 개선안을 마련하여 한 달에 8회에서 0건으로 만들었습니다.',
 '이때 통신비 납부 지연으로 인해 발생한 수수료 300원을 납부 방법변경으로 미발생하게 하였습니다.',
 '우리나라는 해외에 비해 친환경 에너지를 사용하기에 기술적, 지형적문제가 있습니다.',
 '이를 극복하기위한 방면으로는 그린 뉴딜 정책이 가장 적합하다고 생각하며 한국에 맞는 운영을 할 때 더욱 효율적인 운영이 필요하고 이를 위해서 디지털화가 중심이 되어야 한다고 생각합니다.',
 '디지털 뉴딜을 통해 공공일자리를 넘어 숙련된 질높은 일자리를 구축하는 등 구조적 변화를 해야합니다.',
 '이를 시행할 때 있어서 무엇보다 안전한 전력수급이 중요합니다.',
 '이를 위해서는 전력과 디지털 분야 두 분야에 특화된 전력관련한 디지털 지식을 가지고 숙련된 인재를 양성하고 대기시키는 게 중요하고 기존 설비에 대해서는 주기적인 검사를 통해 예방을 해야합니다.',
 '따라서 향후에 디지털 뉴딜을 중심으로 지속가능한 미래를 만들도록 미래사업에 투입될 인원을 양성하는 업무를 해나가고 싶습니다.',
 '사무행정원으로서 해외의 사례들과 우리나라의 현황을 비교하고 분석하면서 그린 경제로의 전

In [54]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor(
    min_frequency=10,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)

word_extractor.train(corpus)  # list of str or like

training was done. used memory 0.610 Gby 0.608 Gb


In [56]:
words = word_extractor.extract()
print(len(words))
print(words['하전'])

all cohesion probabilities was computed. # words = 413083
all branching entropies was computed # words = 243445
all accessor variety was computed # words = 243445
202421


KeyError: '하전'

In [86]:
from soynlp.tokenizer import LTokenizer

cohesion_score = {word:score.cohesion_forward for word, score in words.items()}
cohesion_score["한전KPS"] = 1.0
cohesion_score["한전 KPS"] = 1.0
cohesion_score["한전"] = 1.0
tokenizer = LTokenizer(scores=cohesion_score)

sent = "멋진 한전직원이 되고싶습니다"
print(tokenizer.tokenize(sent))

['멋진', '한전', '직원이', '되고싶습니다']


---

In [149]:
with open("./jasa_dict_0910.json", "r") as f:
    base_dict_0910 = json.load(f)
corpus = []
for k_1, v_1 in base_dict_0910.items():
    for k_2, v_2 in base_dict_0910[k_1].items():
        for k_3, v_3 in base_dict_0910[k_1][k_2].items():
            corpus += [v_3]
            
corpus

['글로벌 환경에서의 성장기회 한전KPS에 지원하게 된 동기는 회사의 해외 대규모 프로젝트와 글로벌 사업 확장을 통해 국제적으로 활동할 수 있는 기회를 제공받을 수 있다는 점에서 비롯되었습니다.',
 '한전KPS가 중동, 아프리카, 동남아시아 등 다양한 국가에서 전력 설비 유지보수와 정비를 수행하는 모습을 보면서, 이러한 글로벌 환경에서 일하며 다양한 문화와 협력할 수 있는 기회를 갖고자 했습니다.',
 'NGO 활동을 통해 다양한 외부 기관과 협력하여 프로젝트를 성공적으로 수행한 경험이 있습니다.',
 '의료 낙후 지역의 고등학생들을 대상으로 건강 불평등 문제를 해결하기 위해 강원도 교육청과 강원도 치과의사회와 협력하여 캠프를 기획하고 실행했습니다.',
 '이 과정에서 각 기관의 요구사항을 조율하고, 융통성 있는 계획을 마련하며 협력을 이끌어냈습니다.',
 '이러한 경험을 통해 다양한 이해관계자와의 소통과 협력, 효율적인 자원 관리, 문제 해결 능력 등 글로벌 프로젝트 수행에 필요한 중요한 역량을 키웠습니다.',
 '국내외 대규모 인프라 프로젝트를 통한 전문성 개발 먼저 국내 대규모 인프라 프로젝트와 관련된 업무를 수행하며 전문 지식을 쌓고자 합니다.',
 '국내에서의 업무 경험을 통해 전력 설비의 유지보수와 정비에 대한 깊이 있는 이해를 높이고, 재무 및 회계 관리, 계약 관리, 프로젝트 관리 기술 등의 실무적인 능력을 갖출 것 입니다.',
 '이를 통해 한전KPS의 국내 인프라 프로젝트에서 핵심적인 역할을 수행할 수 있도록 준비하겠습니다.',
 '국내의 경험과 지식을 바탕으로, 국제적인 무대에서도 업무를 수행하고 싶습니다.',
 '해외 프로젝트를 통해 다양한 국가와 협력하며 전문성을 더욱 발전시킬 것입니다.',
 '현지 정부, 기업, 기관들과의 협력을 통해 상호 신뢰를 구축하고, 장기적인 파트너십을 형성하며, 글로벌 전력 설비 유지보수의 선두주자로서 한전KPS의 위상을 높이는 데 기여하고자 합니다.',
 '결론적으로, 한전KPS에서 국내 대규모 인프라 프

In [150]:
word_extractor = WordExtractor(
    min_frequency=1,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)

word_extractor.train(corpus)  # list of str or like
words = word_extractor.extract()

training was done. used memory 1.202 Gbse memory 1.227 Gb
all cohesion probabilities was computed. # words = 413083
all branching entropies was computed # words = 243445
all accessor variety was computed # words = 243445


In [158]:
print(len(words))
print(words['한전k'])

202421
Scores(cohesion_forward=0.17311025179781844, cohesion_backward=0, left_branching_entropy=2.7325343185507407, right_branching_entropy=-0.0, left_accessor_variety=35, right_accessor_variety=1, leftside_frequency=657, rightside_frequency=0)


In [168]:
from soynlp.tokenizer import LTokenizer

cohesion_score = {word:score.cohesion_forward for word, score in words.items()}
# cohesion_score["한전KPS"] = 1.0
# cohesion_score["한전 KPS"] = 1.0
cohesion_score["한전k"] = 1.0
tokenizer = LTokenizer(scores=cohesion_score)

sent = "멋진 한전k 직원이 되고싶습니다"
print(tokenizer.tokenize(sent))

['멋진', '한전k', '직원이', '되고싶습니다']
